# Data Management: Citi Bike 2016 Dataset

## Download Data

In [ ]:
# import packages
import os
import sys
import numpy as np
import pandas as pd

In [4]:
# import the getCitiBikeCSV function from FBB
def getCitiBikeCSV(datestring):
    '''Downloads citibike data and unzips it.
       If the data is downloaded by not unzippeds it zips it.
       Moves the data to $PUIDATA
    Arguments:
        date string as yyyymm
    '''
    print ("Downloading", datestring)
    ### First I will heck that it is not already there
    if not os.path.isfile(os.getenv("PUIDATA") + "/" + datestring + "-citibike-tripdata.csv"):
        if os.path.isfile(datestring + "-citibike-tripdata.csv"):
            # if in the current dir just move it
            if os.system("mv " + datestring + "-citibike-tripdata.csv " + os.getenv("PUIDATA")):
                print ("Error moving file!, Please check!")
        #otherwise start looking for the zip file
        else:
            if not os.path.isfile(os.getenv("PUIDATA") + "/" + datestring + "-citibike-tripdata.zip"):
                if not os.path.isfile(datestring + "-citibike-tripdata.zip"):
                    os.system("curl -O https://s3.amazonaws.com/tripdata/" + datestring + "-citibike-tripdata.zip")
                ###  To move it I use the os.system() functions to run bash commands with arguments
                os.system("mv " + datestring + "-citibike-tripdata.zip " + os.getenv("PUIDATA"))
            ### unzip the csv 
            os.system("unzip " + os.getenv("PUIDATA") + "/" + datestring + "-citibike-tripdata.zip")
            ## NOTE: old csv citibike data had a different name structure. 
            if '2014' in datestring:
                os.system("mv " + datestring[:4] + '-' +  datestring[4:] + 
                          "\ -\ Citi\ Bike\ trip\ data.csv " + datestring + "-citibike-tripdata.csv")
            os.system("mv " + datestring + "-citibike-tripdata.csv " + os.getenv("PUIDATA"))
    ### One final check:
    if not os.path.isfile(os.getenv("PUIDATA") + "/" + datestring + "-citibike-tripdata.csv"):
        print ("WARNING!!! something is wrong: the file is not there!")

    else:
        print ("file in place, you can continue")

In [2]:
# download Citi Bike data in 2016
for i in range(12):
    datestring = str(201601 + i)
    getCitiBikeCSV(datestring)

file in place, you can continue
file in place, you can continue
file in place, you can continue
file in place, you can continue
file in place, you can continue
file in place, you can continue
file in place, you can continue
file in place, you can continue
file in place, you can continue
file in place, you can continue
file in place, you can continue
file in place, you can continue


## Generate station data

In [116]:
# use latest data (2016/12) to obtain unique stations
CB = pd.read_csv(os.getenv("PUIDATA") + "/" + '201612' + "-citibike-tripdata.csv")
CB.columns

Index(['Trip Duration', 'Start Time', 'Stop Time', 'Start Station ID',
       'Start Station Name', 'Start Station Latitude',
       'Start Station Longitude', 'End Station ID', 'End Station Name',
       'End Station Latitude', 'End Station Longitude', 'Bike ID', 'User Type',
       'Birth Year', 'Gender'],
      dtype='object')

In [55]:
# extract data from non-duplicated start stations
stations = CB.drop_duplicates('Start Station ID')
stations.sort_values('Start Station ID', inplace=True)
stations = stations.iloc[:,range(3,7)]
stations['Coordinates'] = list(zip(stations['Start Station Latitude'], stations['Start Station Longitude']))
stations.head()

/Users/Yuwen/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,Start Station ID,Start Station Name,Start Station Latitude,Start Station Longitude,Coordinates
200,72,W 52 St & 11 Ave,40.767272,-73.993929,"(40.76727216, -73.99392888)"
15,79,Franklin St & W Broadway,40.719116,-74.006667,"(40.71911552, -74.00666661)"
5037,82,St James Pl & Pearl St,40.711174,-74.000165,"(40.71117416, -74.00016545)"
80,83,Atlantic Ave & Fort Greene Pl,40.683826,-73.976323,"(40.68382604, -73.97632328)"
137,116,W 17 St & 8 Ave,40.741776,-74.001497,"(40.74177603, -74.00149746)"


In [33]:
# export csv to Carto for query analysis
stations.to_csv('stations.csv')

## Identify stations within 1 km of the Barclay Center

In [34]:
# import packages (compatible between py2 and py3)
try:
    import urllib2
    from urllib2 import HTTPError
    from urllib2 import urlopen
    from urllib import urlencode
    from StringIO import StringIO as io

except ImportError:
    import urllib
    from urllib.error import HTTPError
    from urllib.request import urlopen, Request
    from urllib.parse import urlencode
    from io import BytesIO as io

import ast
import pandas as pd

In [35]:
# set SQL source
SQL_SOURCE = 'https://ywc249.carto.com/api/v2/sql?q='

# define query function
def queryCartoDB(query, format='CSV', source=SQL_SOURCE):
    '''queries carto datasets from a given carto account
    Arguments: 
    query - string: a valid sql query string
    format - outlut format  OPTIONAL (default CSV)
    source - a valid sql api endpoint OPTIONAL (default carto fb55 account)
    Returns:
    the return of the sql query AS A STRING
    NOTES:
    designed for the carto API, tested only with CSV return format'''
    
    data = urlencode({'format':format, 'q':query}).encode('utf-8')
    try:
        response = urlopen(source, data)
    except HTTPError as e:
        raise ValueError('\n'.join(ast.literal_eval(e.readline())['error']))
    except Exception:
        raise
    return response.read()

In [76]:
# select stations within 1 km of the Barclay Center
q1 = '''
SELECT
    start_station_id,
    CDB_TransformToWebmercator(CDB_LatLng(
        start_station_latitude,
        start_station_longitude
        )
) as the_geom_webmercator, MIN(cartodb_id) as cartodb_id
FROM stations
WHERE
     ST_DWithin(CDB_LatLng(
         start_station_latitude,
         start_station_longitude
)::geography, CDB_LatLng(40.683048,-73.975961)::geography, 1000)
GROUP BY
      start_station_id,
      start_station_latitude,
      start_station_longitude
'''
nearby = pd.read_csv(io(queryCartoDB(q1)), sep=',')
nearby = list(nearby['start_station_id'])
nearby.sort()

## Subset and concatenate the major dataframe for analysis
- Subset trips beginning from the 32 stations near the Barclay Center
- Subset trips beginning between 16:00 - 24:00

In [118]:
# examine variables
print(CB.columns)
CB2 = pd.read_csv(os.getenv("PUIDATA") + "/" + '201601' + "-citibike-tripdata.csv")
print(CB2.columns)

Index(['Trip Duration', 'Start Time', 'Stop Time', 'Start Station ID',
       'Start Station Name', 'Start Station Latitude',
       'Start Station Longitude', 'End Station ID', 'End Station Name',
       'End Station Latitude', 'End Station Longitude', 'Bike ID', 'User Type',
       'Birth Year', 'Gender'],
      dtype='object')
Index(['tripduration', 'starttime', 'stoptime', 'start station id',
       'start station name', 'start station latitude',
       'start station longitude', 'end station id', 'end station name',
       'end station latitude', 'end station longitude', 'bikeid', 'usertype',
       'birth year', 'gender'],
      dtype='object')


In [117]:
# subset data by month and append as a list of 12 dfs
CB2016 = []
CBshape = []

for i in range(12):
    datestring = str(201601 + i)
    df = pd.read_csv(os.getenv("PUIDATA") + "/" + datestring + "-citibike-tripdata.csv")
    #df.columns = [c.lower() for c in df.columns]
    df.columns = ['tripduration', 'starttime', 'stoptime',
                  'start station id', 'start station name', 'start station latitude', 'start station longitude',
                  'end station id', 'end station name', 'end station latitude', 'end station longitude',
                  'bikeid', 'usertype', 'birth year', 'gender']
    df = df[df['start station id'].apply(lambda x: x in nearby)]
    df['starttime'] = pd.to_datetime(df['starttime'])
    df = df[df['starttime'].dt.hour >= 16]
    CB2016.append(df)
    CBshape.append(df.shape)

In [130]:
# concatenate data from all 12 months and validate merge
CB_concat = pd.concat(CB2016, axis=0)
CB_concat.reset_index(inplace=True)
CB_concat = CB_concat.drop('index', axis=1)

print('Total trips for 12 months: %s\nMerged shape: %s'%(sum([s[0] for s in CBshape]), CB_concat.shape))
CB_concat.head()

Total trips for 12 months: 100666
Merged shape: (100666, 15)


,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender
0,173,2016-01-01 16:03:10,1/1/2016 16:06:03,243,Fulton St & Rockwell Pl,40.688226,-73.979382,241,DeKalb Ave & S Portland Ave,40.689810,-73.974931,23976,Subscriber,1971.0,2
1,136,2016-01-01 16:05:54,1/1/2016 16:08:11,420,Clermont Ave & Lafayette Ave,40.687645,-73.969689,270,Adelphi St & Myrtle Ave,40.693083,-73.971789,17362,Subscriber,1980.0,1
2,616,2016-01-01 16:08:41,1/1/2016 16:18:58,262,Washington Park,40.691782,-73.973730,397,Fulton St & Clermont Ave,40.684157,-73.969223,22865,Subscriber,1981.0,1
3,653,2016-01-01 16:13:47,1/1/2016 16:24:40,83,Atlantic Ave & Fort Greene Pl,40.683826,-73.976323,278,Concord St & Bridge St,40.697666,-73.984764,16270,Subscriber,1976.0,1
4,659,2016-01-01 16:13:47,1/1/2016 16:24:46,83,Atlantic Ave & Fort Greene Pl,40.683826,-73.976323,278,Concord St & Bridge St,40.697666,-73.984764,22909,Subscriber,1985.0,2


In [131]:
# export csv file
CB_concat.to_csv('CB2016_subset.csv')